# Trees: Binary Trees (BT)
It's a dictionary in an Abstract Data Structure (ADT)

## Create Binary-Tree
It's a Heap, but where a Heap isn't concerned with it's value relative to it's siblings (only its parent), a BST is concerned with it's value relative to it's siblings and it's parent.

1. Exists as an Array or Linked List.
2. Search: Big-Oh(N)

In [ ]:
def bst_search(tree, target):
    i = 0
    current = tree[i]
    while current:
        left = 2*i+1
        right = 2*i+1
        if target == current:
            return True
        if current > target:
            current = tree[left]
        elif current < target:
            current = tree[right]
        i += 1
    return False

2. cont'd | We're looking for a target value by iterating through the tree's branches looking for the target value.
3. Insert: Big-Oh(Log(n)) | However, we wouldn't want to store a BST into an Array if we're going to be implementing Insert capabilities, since inserting into a list, has the __worst case time__ of Big-Oh(n): if we insert into index 0, then we have to shift n elements to the right. So we should store a BST into a _Linked List_ if we're allowing insert operations since moving pointers can be done in Theta(1) time, and Big-Oh(Log(n)) time to find the right spot to insert.

In [59]:
from collections import deque as Queue

class TreeNode:
    def __init__(self, value: any):
        self.value = value
        self.left = None
        self.right = None

    def level_order_traversal(self):
        q = Queue([self])
        while q:
            level_nodes = []
            for i in range(0, len(q)):
                node = q.popleft()
                if node.left:
                    q.append(node.left)
                if node.right:
                    q.append(node.right)
                level_nodes.append(node.value)
            print(level_nodes)

In [60]:
def insert(root: TreeNode = None, v: any = None):
    new_node = TreeNode(v)
    if not root:
        return new_node
    current = root
    parent = root
    while current:
        parent = current
        if new_node.value == current.value:
            return False # Already exists: can't have duplicates
        if new_node.value < current.value:
            current = current.left
        else:
            current = current.right
    if parent.value < new_node.value:
        parent.right = new_node
    else:
        parent.left = new_node
    return root

def build_bst(values):
    root = None
    for v in values:
        root = insert(root, v)
    return root
root = build_bst([7, 5, 9, 8, 6, 10, 3, 1])

In [61]:
root.level_order_traversal()

[7]
[5, 9]
[3, 6, 8, 10]
[1]


* This method of inserting into a tree creates a one-degree tree.  If we want to create a tree with an even distribution of nodes across all levels, then we need a _Queue_.
* We'll expect to be given a list of values, and should be inserted into the tree.
* We'll use the typical left-child, right-child index algorithm to assign them to their respective locations.
    - 2 * i + 1 = Left Child to the i'th Node
    - 2 * i + 2 = Right Child to the i'th Node

In [66]:
from collections import deque as queue


def build_a_bst_from_sorted_list(values):
    root = TreeNode(values[0])
    q = queue()
    q.append(root)
    i = 0
    while q:
        node = q.popleft()
        li = 2 * i + 1
        ri = 2 * i + 2
        if li < len(values):
            node.left = TreeNode(values[li])
            q.append(node.left)
        if ri < len(values):
            node.right = TreeNode(values[ri])
            q.append(node.right)
        i += 1
    return root
root = build_a_bst_from_sorted_list([1, 2, 3, 4, 5, 6, 7])
root.level_order_traversal()
print('\n----------')
root = build_a_bst_from_sorted_list([7, 5, 9, 8, 6, 10, 3])
root.level_order_traversal()

[1]
[2, 3]
[4, 5, 6, 7]

----------
[7]
[5, 9]
[8, 6, 10, 3]


* The approach above will only work if the input is a sorted list. It will create as perfectly balanced a Binary tree as possible. It should be noted this method is using the same technique we use to traverse as BFS.

------------------------------------

## Create Binary-Tree from Lists
* We can create a Binary-Tree from a list of pre-order, and in-order lists.
* _Pre-Order_ tells us the sequence by which we should create each node.
* _In-Order_ tells us which node's are left and right children of each node.
* **Control Flow**
    1. Pick the left-most node in from the _Pre-Order_ list. This will be the next root we build.
    2. Search the _In-Order_ list for the value from step 1.
    3. Once we have the index from the _In-Order_ list we can look to 1-index for the left child, and 1+index for the right child.
        - If the index 1 unit away does not exist, then we know that child is not defined.
    4. We use the same setup as Level-Order traversal; using a Queue to enqueue nodes after they're created. The Intuition is that once a node is created, it's immediately attached to it's parent and then enqueued. Once it's de-queued it's children are attached to it, and we no longer need to worry about that particular node.

In [56]:
def memoize(memo, some_list):
    map = {}
    for i, val in enumerate(some_list):
        map[val] = i
    return map

def build_tree_from_traversal_lists(pre_order, in_order, io_map=None, pre_start=0, pre_end=None, in_start=0, in_end=None):
    if io_map is None:
        io_map = memoize(io_map, in_order)
        pre_end = len(pre_order) - 1
        in_end = len(in_order) - 1
    if pre_start > pre_end: return
    v = pre_order[pre_start]
    if pre_start == pre_end:
        return TreeNode(v)  # leaf
    root_ix = io_map.get(v)
    count = root_ix - in_start
    root = TreeNode(v)
    root.left = build_tree_from_traversal_lists(pre_order, in_order, io_map,
                                                pre_start + 1, pre_start + count, in_start, root_ix - 1)
    root.right = build_tree_from_traversal_lists(pre_order, in_order, io_map,
                                                 pre_start + count + 1, pre_end, root_ix + 1, in_end)
    return root
root = build_tree_from_traversal_lists(pre_order=[3, 9, 20, 15, 7], in_order=[9, 3, 15, 20, 7])
root.level_order_traversal()
print('\n---------')
root = build_tree_from_traversal_lists(pre_order=[8, 5, 3, 2, 10, 9, 12], in_order=[2, 3, 5, 8, 9, 10, 12])
root.level_order_traversal()

[3]
[9, 20]
[15, 7]

---------
[8]
[5, 10]
[3, 9, 12]
[2]
